In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Path to the directory containing training and testing data
train_dir = '/kaggle/input/cutmix-17march/cutmix_new/train'
test_dir = '/kaggle/input/cutmix-17march/cutmix_new/test'

# Image dimensions
img_height = 299
img_width = 299
batch_size = 32

# Preprocessing function for InceptionV3
preprocess_input = tf.keras.applications.inception_v3.preprocess_input

# Create data generators for training and testing
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# Load the pre-trained InceptionV3 model without the top classification layer
inception_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the layers of the pre-trained InceptionV3
for layer in inception_base.layers:
    layer.trainable = False

# Add custom classification layers on top of InceptionV3
x = layers.GlobalAveragePooling2D()(inception_base.output)
x = layers.Dense(256, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

# Create the model
model = models.Model(inputs=inception_base.input, outputs=output)

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=None,  # Set to None to run until the generator stops
                    epochs=10,
                    validation_data=test_generator,
                    validation_steps=None)  # Set to None to run until the generator stops

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print("Test Accuracy:", accuracy)


2024-03-17 13:11:13.444317: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 13:11:13.444462: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 13:11:13.627224: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 1294 images belonging to 2 classes.
Found 856 images belonging to 2 classes.
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


41/41 ━━━━━━━━━━━━━━━━━━━━ 349s 8s/step - accuracy: 0.5469 - loss: 0.9098 - val_accuracy: 0.7453 - val_loss: 0.5178
Epoch 2/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 325s 8s/step - accuracy: 0.7476 - loss: 0.5320 - val_accuracy: 0.8306 - val_loss: 0.4117
Epoch 3/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 404s 8s/step - accuracy: 0.7814 - loss: 0.4564 - val_accuracy: 0.8551 - val_loss: 0.3704
Epoch 4/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 368s 8s/step - accuracy: 0.8071 - loss: 0.3871 - val_accuracy: 0.8458 - val_loss: 0.3567
Epoch 5/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 374s 8s/step - accuracy: 0.8719 - loss: 0.3244 - val_accuracy: 0.8493 - val_loss: 0.3460
Epoch 6/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 382s 8s/step - accuracy: 0.8720 - loss: 0.3213 - val_accuracy: 0.8621 - val_loss: 0.3241
Epoch 7/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 389s 8s/step - accuracy: 0.8806 - loss: 0.2862 - val_accuracy: 0.8563 - val_loss: 0.3204
Epoch 8/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 375s 8s/step - accuracy: 0.9004 - loss: 0.2538 - val_accuracy: 0.8703 - val_loss: 0.

In [2]:
image_path = '/kaggle/input/monalisa-predict-cutmix/monalisa.jpeg'

img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

prediction = model.predict(img_array)

if prediction < 0.5:
    print("AI generated Image")
else:
    print("Original Image")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Original Image
